In [ ]:
<h3 style="text-align: center;">Your Recommended Medicines are shown below: </h3>
		<div class="row">
			{% for product in gained_products %}
				<div class="col-lg-4 product-box">
					<img class="thumbnail" src="{{ product.get_imageURL }}">
					<div class="box-element">
						<h6><strong>{{product.name}}</strong></h6>
						<hr>
	
						<button  class="update_cart btn btn-outline-secondary add-btn" product_id ="{{product.id}}">Add to Cart</button>
	
						<a class="btn btn-outline-success" href="{% url 'item_detail' product.id %}">View</a>
						<h4 style="display: inline-block; float: right"><strong>&dollar;{{product.price}}</strong></h4>
	
					</div>
				</div>
			{% endfor %}
		</div>

In [ ]:
def create_matrix(df):
    
    N = len(df['user_id'].unique())
    M = len(df['product_id'].unique())

    # Map Ids to indices
    user_mapper = dict(zip(np.unique(df["user_id"]), list(range(N))))
    product_mapper = dict(zip(np.unique(df["product_id"]), list(range(M))))

    # Map indices to IDs
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["user_id"])))
    product_inv_mapper = dict(zip(list(range(M)), np.unique(df["product_id"])))

    user_index = [user_mapper[i] for i in df['user_id']]
    product_index = [product_mapper[i] for i in df['product_id']]

    csr = csr_matrix((df["rating"], (product_index, user_index)), shape=(M, N))

    return csr, user_mapper, product_mapper, user_inv_mapper, product_inv_mapper

In [ ]:
def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
	
	neighbour_ids = []
	
	movie_ind = movie_mapper[movie_id]
	movie_vec = X[movie_ind]
	k+=1
	kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
	kNN.fit(X)
	movie_vec = movie_vec.reshape(1,-1)
	neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)
	for i in range(0,k):
		n = neighbour.item(i)
		neighbour_ids.append(movie_inv_mapper[n])
	neighbour_ids.pop(0)
	return neighbour_ids


movie_titles = dict(zip(movies['movieId'], movies['title']))

movie_id = 7

similar_ids = find_similar_movies(movie_id, X, k=10)
movie_title = movie_titles[movie_id]

print(f"Since you watched:- {movie_title}")
for i in similar_ids:
	print(movie_titles[i])

In [ ]:
def recommend_movies_for_user(user_id, X, k=10):
	df1 = ratings[ratings['userId'] == user_id]
	
	if df1.empty:
		print(f"User with ID {user_id} does not exist.")
		return

	movie_id = df1[df1['rating'] == max(df1['rating'])]['movieId'].iloc[0]

	movie_titles = dict(zip(movies['movieId'], movies['title']))

	similar_ids = find_similar_movies(movie_id, X, k)
	movie_title = movie_titles.get(movie_id, "Movie not found")

	if movie_title == "Movie not found":
		print(f"Movie with ID {movie_id} not found.")
		return

	print(f"Since you watched {movie_title}, you might also like:\n")
	for i in similar_ids:
		print(movie_titles.get(i, "Movie not found"))
user_id = 150
recommend_movies_for_user(user_id, X, k=12)